In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, date
from sklearn.metrics import accuracy_score
from kaggle.competitions import twosigmanews
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from itertools import chain

**HELPER FUNCTIONS:**
Here i implement some simple transformers to use with pipeline and helper functions

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
#--------------------------------------------------------------
class OpenCloseTransformer(BaseEstimator, TransformerMixin):
    def __init__ (self,
                  open_col_name = 'open',
                  close_col_name = 'close',
                  new_col = 'fprise',
                  remove_old_col = False
                  ):
        self.openc = open_col_name
        self.closec = close_col_name
        self.newc = new_col
        self.isRemove = remove_old_col
        
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X[self.newc] = (X[self.openc]-X[self.closec])/((X[self.openc]+X[self.closec])/2)
        if self.isRemove == True:
            X = X.drop([self.openc, self.closec], axis=1)
        return X
#--------------------------------------------------------------
class toTypeTransformer(BaseEstimator, TransformerMixin):
    def __init__ (self,
                  cat_list = [],
                  tranformType = 'U'
                  ):
        self.ttype = tranformType
        self.catlist = cat_list
        
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X.loc[:self.catlist] = X[self.catlist].astype(self.ttype)
        return X
#--------------------------------------------------------------    
class dataSelector(BaseEstimator, TransformerMixin):
    def __init__ (self,
                  cat_list = [],
                  isRetDataFrame = False):
        self.catlist = cat_list
        self.isDf = isRetDataFrame
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if(self.isDf == True):
            return X[self.catlist]
        else:
            return X[self.catlist].values
    
#--------------------------------------------------------------
class toLabeEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__ (self,
                  cat = '',
                  create_new_col = True,
                  pref_col = 'le_'):
        
        self.cnc = create_new_col
        self.cat = cat
        self.pref = pref_col
        pass

    def fit(self, X, y=None):
        self.enc = LabelEncoder()
        self.enc.fit(X[self.cat])
        return self

    def transform(self, X, y=None):
        if(True == self.cnc):
            #X[self.pref + self.cat] = self.enc.transform(X[self.cat])
            X.loc[:,self.pref + self.cat] = self.enc.transform(X[self.cat])
        else:
            #X[self.cat] = self.enc.transform(X[self.cat])
            X.loc[:,self.cat] = self.enc.transform(X[self.cat])
        return X
    
    
 #--------------------------------------------------------------
class fillNATransformer(BaseEstimator, TransformerMixin):
    def __init__ (self,
                  _list = [],
                  _value = 'undefine'):
        
        self.lst = _list
        self.val = _value
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X.loc[:,self.lst] = X[self.lst].fillna(self.val)
        return X   
#--------------------------------------------------------------
class fillNASrcDscTransformer(BaseEstimator, TransformerMixin):
    def __init__ (self,
                  _list_src = [],
                  _list_dst = []):
        
        self.src = _list_src
        self.dst = _list_dst
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        #X[self.src] = X[self.src].fillna(0.0).astype(np.float64)
        X.loc[:,self.src] = X[self.src].fillna(0.0).astype(np.float64)
        for i in range(len(self.src)):
            #X[self.dst[i]] = X[self.dst[i]].fillna(X[self.src[i]])
            X.loc[:,self.dst[i]] = X[self.dst[i]].fillna(X[self.src[i]])
        return X
#--------------------------------------------------------------------------------
def adjust_time(market_train_df,news_train_df,time_threshold=22):

    # Split date into before and after 22h (the time used in train data)
    # E.g: 2007-03-07 23:26:39+00:00 -&gt; 2007-03-08 00:00:00+00:00 (next day)
    #      2009-02-25 21:00:50+00:00 -&gt; 2009-02-25 00:00:00+00:00 (current day)
    news_train_df['time'] = (news_train_df['time'] - np.timedelta64(time_threshold,'h')).dt.ceil('1D')
    # Round time of market_train_df to 0h of curret day
    market_train_df['time'] = market_train_df['time'].dt.floor('1D')
    return (market_train_df,news_train_df)
#--------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
def trainTestSplit(x_all, y_all, _train, rs = 42):
    
    t_i, _i = train_test_split(_train.index.values,test_size=0.10, random_state=rs)
    te_i, v_i = train_test_split(_i,test_size=0.40, random_state=rs)
    
    tr_x = x_all[t_i]
    tr_r = y_all[t_i]
    tr_y = np.int32(y_all[t_i] > 0.0)

    v_x = x_all[v_i]
    v_r = y_all[v_i]
    v_y = np.int32(y_all[v_i] > 0.0)
    v_u = _train.loc[v_i, 'universe']
    v_d = _train.loc[v_i, 'time']

    te_x = x_all[te_i]
    te_r = y_all[te_i]
    te_y = np.int32(y_all[te_i] > 0.0)
    te_u = _train.loc[te_i, 'universe']
    te_d = _train.loc[te_i, 'time']
    
    return (tr_x, tr_y, v_x, v_y, v_u, v_r, v_d, te_x, te_y, te_u, te_d, te_r)
    #tr_x, r_x, tr_y, r_y = train_test_split(x_all, y_all, test_size=0.10, random_state=rs)
    #te_x, v_x, te_y, v_y = train_test_split(r_x, r_y, test_size=0.40, random_state=rs+1)
    #return (tr_x, tr_y, te_x, te_y, v_x, v_y)



#--------------------------------------------------------------------------------
def getFeatIndexes(df, cat_feat, num_feat, multp = 0.5, return_lookup_df = False, verbose = 0):
    if cat_feat != []:
        multp = 0.5
        cat_feats_idx = np.array([df.columns.get_loc(x) for x in cat_feat])
        cat_feats_dm = np.array([np.int16(len(df[x].unique()) **multp) for x in cat_feat])
        cat_feat_lookup = pd.DataFrame({'feature': cat_feat, 'column_index': cat_feats_idx, 'column_dm': cat_feats_dm})
    else:
        cat_feats_idx = np.array([])
        cat_feats_dm = np.array([])
        cat_feat_lookup = pd.DataFrame({'feature': cat_feat, 'column_index': cat_feats_idx, 'column_dm': cat_feats_dm})
        
    num_feats_idx = np.array([df.columns.get_loc(x) for x in num_feat])
    num_feat_lookup = pd.DataFrame({'feature': num_feat, 'column_index': num_feats_idx})

    if(verbose > 0):
        display(cat_feat_lookup.head())
        display(num_feat_lookup.head())
    
    if return_lookup_df == True:
        return cat_feats_idx, cat_feats_dm, num_feats_idx, cat_feat_lookup, num_feat_lookup
    
    return cat_feats_idx, cat_feats_dm, num_feats_idx

#----------------------------------------------------------------------------
def change_date_to_datetime(x):
    str_time = str(x)
    date = '{}-{}-{}'.format(str_time[:4], str_time[5:7], str_time[8:10])
    return date

#-----------------------------------------------------------------------------
def join_market_news(market_train_df, news_train_df):
    # Fix asset codes (str -> list)
    news_train_df['assetCodes'] = news_train_df['assetCodes'].str.findall(f"'([\w\./]+)'")    
    
    # Expand assetCodes
    assetCodes_expanded = list(chain(*news_train_df['assetCodes']))
    assetCodes_index = news_train_df.index.repeat( news_train_df['assetCodes'].apply(len) )

    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})

    # Create expandaded news (will repeat every assetCodes' row)
    news_cols = ['time', 'assetCodes'] + sorted(news_cols_agg.keys())
    news_train_df_expanded = pd.merge(df_assetCodes, news_train_df[news_cols], left_on='level_0', right_index=True, suffixes=(['','_old']))

    # Free memory
    del news_train_df, df_assetCodes

    # Aggregate numerical news features
    news_train_df_aggregated = news_train_df_expanded.groupby(['time', 'assetCode']).agg(news_cols_agg)
    
    # Free memory
    del news_train_df_expanded

    # Convert to float32 to save memory
    news_train_df_aggregated = news_train_df_aggregated.apply(np.float32)

    # Flat columns
    news_train_df_aggregated.columns = ['_'.join(col).strip() for col in news_train_df_aggregated.columns.values]

    # Join with train
    market_train_df = market_train_df.join(news_train_df_aggregated, on=['time', 'assetCode'])

    # Free memory
    del news_train_df_aggregated
    
    return market_train_df

### Loading market data and news

In [ ]:
#create enviroment
env = twosigmanews.make_env()

In [ ]:
(market_train, news_train) = env.get_training_data()

In [ ]:
display(market_train.head(2))
display(news_train.head(2))

In [ ]:
_date = date(2010, 1, 1)
adjust_time(market_train, news_train)
news_train['time'] = news_train['time'].dt.date
market_train['time'] = market_train['time'].dt.date
market_train = market_train.loc[market_train['time']>=_date]
news_train = news_train.loc[news_train['time']>=_date]

In [ ]:
market_train = market_train.reset_index()

In [ ]:
news_cols_agg = {
#    'urgency': ['min', 'count'],
#    'takeSequence': ['max'],
#    'bodySize': ['min', 'max', 'mean', 'std'],
#    'wordCount': ['min', 'max', 'mean', 'std'],
#    'sentenceCount': ['min', 'max', 'mean', 'std'],
#    'companyCount': ['min', 'max', 'mean', 'std'],
#    'marketCommentary': ['min', 'max', 'mean', 'std'],
#    'relevance': ['min', 'max', 'mean', 'std'],
#    'sentimentNegative': ['min', 'max', 'mean', 'std'],
#    'sentimentNeutral': ['min', 'max', 'mean', 'std'],
#    'sentimentPositive': ['min', 'max', 'mean', 'std'],
#    'sentimentWordCount': ['min', 'max', 'mean', 'std'],
#    'noveltyCount12H': ['min', 'max', 'mean', 'std'],
#    'noveltyCount24H': ['min', 'max', 'mean', 'std'],
#    'noveltyCount3D': ['min', 'max', 'mean', 'std'],
#    'noveltyCount5D': ['min', 'max', 'mean', 'std'],
#    'noveltyCount7D': ['min', 'max', 'mean', 'std'],
#    'volumeCounts12H': ['min', 'max', 'mean', 'std'],
#    'volumeCounts24H': ['min', 'max', 'mean', 'std'],
#    'volumeCounts3D': ['min', 'max', 'mean', 'std'],
#    'volumeCounts5D': ['min', 'max', 'mean', 'std'],
#    'volumeCounts7D': ['min', 'max', 'mean', 'std']
    'urgency': ['min', 'count'],
    'takeSequence': ['max'],
    'bodySize': ['mean'],
    'wordCount': ['mean'],
    'sentenceCount': ['mean'],
    'companyCount': ['mean'],
    'marketCommentary': ['mean'],
    'relevance': ['mean'],
    'sentimentNegative': ['mean'],
    'sentimentNeutral': ['mean'],
    'sentimentPositive': ['mean'],
    'sentimentWordCount': ['mean'],
    'noveltyCount12H': ['mean'],
    'noveltyCount24H': ['mean'],
    'noveltyCount3D':  ['mean'],
    'noveltyCount5D':  ['mean'],
    'noveltyCount7D':  ['mean'],
    'volumeCounts12H': ['mean'],
    'volumeCounts24H': ['mean'],
    'volumeCounts3D':  ['mean'],
    'volumeCounts5D':  ['mean'],
    'volumeCounts7D':  ['mean']    
}
market_train = join_market_news(market_train, news_train)

In [ ]:
cat_cols = ['assetCode']
cat_cols_sel = ['le_assetCode'] #assset column name after label encoding
num_cols = ['volume', 
            'op',
            #'close', 'open', 
            'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1',
                    'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 'returnsClosePrevMktres10',
                    'returnsOpenPrevMktres10',
      'urgency_min',
       'urgency_count', 'takeSequence_max', 'bodySize_mean',
       'wordCount_mean', 'sentenceCount_mean', 'companyCount_mean',
       'marketCommentary_mean', 'relevance_mean',
       'sentimentNegative_mean', 'sentimentNeutral_mean',
       'sentimentPositive_mean', 'sentimentWordCount_mean',
       'noveltyCount12H_mean', 'noveltyCount24H_mean',
       'noveltyCount3D_mean', 'noveltyCount5D_mean',
       'noveltyCount7D_mean', 'volumeCounts12H_mean',
       'volumeCounts24H_mean', 'volumeCounts3D_mean',
       'volumeCounts5D_mean', 'volumeCounts7D_mean']

column_market = ['returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
column_raw = ['returnsClosePrevRaw1', 'returnsOpenPrevRaw1','returnsClosePrevRaw10', 'returnsOpenPrevRaw10']

num_cols_fna = ['volume', 
           
        'close', 'open', 
        'urgency_min',
       'urgency_count', 'takeSequence_max', 'bodySize_mean',
       'wordCount_mean', 'sentenceCount_mean', 'companyCount_mean',
       'marketCommentary_mean', 'relevance_mean',
       'sentimentNegative_mean', 'sentimentNeutral_mean',
       'sentimentPositive_mean', 'sentimentWordCount_mean',
       'noveltyCount12H_mean', 'noveltyCount24H_mean',
       'noveltyCount3D_mean', 'noveltyCount5D_mean',
       'noveltyCount7D_mean', 'volumeCounts12H_mean',
       'volumeCounts24H_mean', 'volumeCounts3D_mean',
       'volumeCounts5D_mean', 'volumeCounts7D_mean']


### Create piplines

In [ ]:

data_prep_pipeline = Pipeline([
    ('label_enc', toLabeEncoderTransformer(cat='assetCode')),
    ('fillSrcNa', fillNASrcDscTransformer(_list_src=column_raw, _list_dst=column_market)),
    ('fillNa', fillNATransformer(_list=num_cols_fna, _value=0.0)),
    ('openclose', OpenCloseTransformer(open_col_name='open', close_col_name='close', new_col='op'))
])

num_pipeline = Pipeline([    
    ('selector', dataSelector(cat_list=num_cols, isRetDataFrame = True)),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([    
    ('selector', dataSelector(cat_list=cat_cols_sel, isRetDataFrame = True)),
    #('label_bin', OneHotEncoder(sparse=False))
])

comb_pipeline = FeatureUnion(transformer_list=[
    ('cat_pipeline', cat_pipeline),
    ('num_pipeline', num_pipeline)
])

full_pipeline = Pipeline([    
    ('prep', data_prep_pipeline),
    ('comb', comb_pipeline)
])

In [ ]:
market_train.head()

In [ ]:
unic_assetsCode = market_train.assetCode.unique()
unic_assetsCode

In [ ]:
all_x = full_pipeline.fit_transform(market_train)
all_y = market_train['returnsOpenNextMktres10'].clip(-1, 1)
all_y = np.array(all_y).reshape(-1,1)
#scaler = MinMaxScaler()
#all_y = scaler.fit_transform(np.array(all_y).reshape(-1,1))

In [ ]:
val_cnt = 200000
if val_cnt != 0:
    all_x = all_x[:val_cnt]
    all_y = all_y[:val_cnt]

In [ ]:
display(all_x.shape)
display(all_y.shape)

In [ ]:
#we need indexes of categorical and numerical features
#cat_cols = []
all_used_col = cat_cols + num_cols
#all_used_col = num_cols
(cat_feats_idx, cat_feats_dm, num_feats_idx, cat_feat_lookup, 
 num_feat_lookup) = getFeatIndexes(market_train[all_used_col], cat_cols, num_cols, multp = 0.5, 
                                   return_lookup_df= True, verbose=1)

### TENSORFLOW

In [ ]:
import tensorflow as tf
from functools import partial
from datetime import datetime

In [ ]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def random_batch(X_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_indices]
    y_batch = np.array(y_train)[rnd_indices]
    return X_batch, y_batch,   

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

def getValidationError(_p, _r, _u, _d):
    # calculation of actual metric that is used to calculate final score
    x_t_i = _p.ravel() * _r.ravel() * np.array(_u)
    data = {'day' : _d, 'x_t_i' : x_t_i}
    df = pd.DataFrame(data)
    x_t = df.groupby('day').sum().values.flatten()
    mean = np.mean(x_t)
    std = np.std(x_t)
    score_valid = mean / std
    return score_valid    

### DNNClassifaier 

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError
import shutil
#initializer
he_init = tf.contrib.layers.variance_scaling_initializer()

#classifier
class DNNClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, n_hidden_layers=3, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=500, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None, is_sofmax_loss = False, 
                 logdir = log_dir("sigma_log")):
        
        #"""Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self.is_sofmax_loss = is_sofmax_loss
        self._session = None  
        self.logdir = logdir
        
        # parameters for storing model
        self._checkpoint_path = "./tmp/my_deep_model.ckpt"
        self._checkpoint_epoch_path = self._checkpoint_path + ".epoch"
        self._final_model_path = "./final/my_deep_final_model"      
        
        pass
        
    #create normal dnn with dense layers
    def _dnn(self, inputs, dnn_name, _n_hidden_layers, _n_neurons, _dropout_rate, _batch_norm_momentum, _activation ):
        #"""Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(_n_hidden_layers):
            if _dropout_rate:
                inputs = tf.layers.dropout(inputs, _dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, _n_neurons,
                                     kernel_initializer=self.initializer,
                                     name=dnn_name+"_hidden%d" % (layer + 1))
            if _batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=_batch_norm_momentum,
                                                       training=self._training)
            inputs = _activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs    
    
    #create graph
    def _build_graph(self, n_inputs, n_outputs):
        #"""Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X_i = tf.placeholder(tf.float32, [None, n_inputs], name= 'X')
        y = tf.placeholder(tf.int32, shape=[None, n_outputs], name='y')

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        # Define placeholders for the categorical varaibles
        with tf.name_scope("embedding_var"): 
            self.cat_placeholders, self.cat_cardinalities, self.cat_dm = [], [], []
            for idx,dm in zip(cat_feats_idx,cat_feats_dm):
                exec('self.X_cat_{} = tf.placeholder(tf.int32, shape=[None], name=\'X_cat_{}\')'.format(idx, idx))
                exec('self.cat_placeholders.append(self.X_cat_{})'.format(idx))
                self.cat_cardinalities.append(len(np.unique(market_train[all_used_col].iloc[:,idx])))
                self.cat_dm.append(dm)

        # Add embeddings to input
        X = tf.identity(X_i)
        X2 = tf.identity(X_i)[:,:1]
        for feat, card, dm in zip(self.cat_placeholders, self.cat_cardinalities, self.cat_dm):
            X2 = self.embed_and_attach(X2, feat, card, dm, tf.get_default_graph())
        
        dnn_outputs_c = self._dnn(X2, dnn_name = 'X2_input', _n_neurons = 18,
                                  _dropout_rate = self.dropout_rate,
                                  _n_hidden_layers = 2,
                                 _batch_norm_momentum = self.batch_norm_momentum,
                                 _activation = self.activation)
        
            
        #_dnn(self, inputs, dnn_name, _n_hidden_layers, _n_neurons, _dropout_rate, _batch_norm_momentum, _activation )    
        dnn_outputs_1 = self._dnn(X, dnn_name = 'X_input', _n_neurons = self.n_neurons,
                                  _dropout_rate = self.dropout_rate,
                                  _n_hidden_layers = self.n_hidden_layers,
                                 _batch_norm_momentum = self.batch_norm_momentum,
                                 _activation = self.activation)
        dnn_outputs = self._dnn(dnn_outputs_1, dnn_name = 'X_input_2', _n_neurons = 4,
                                _dropout_rate = self.dropout_rate,
                                _n_hidden_layers = 2,
                                _batch_norm_momentum = self.batch_norm_momentum,
                                _activation = self.activation)
        dnn_conc = tf.concat([dnn_outputs, dnn_outputs_c], 1)
        logits = tf.layers.dense(dnn_conc, n_outputs, kernel_initializer=he_init, name="logits")

        
        with tf.name_scope("loss"):    
            if self.is_sofmax_loss == True:
                xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
            else:
                y_as_float = tf.cast(y, tf.float32)
                xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_as_float, logits=logits)

            loss = tf.reduce_mean(xentropy, name='loss')

            loss_v_summary = tf.summary.scalar('validation_loss', loss)
            loss_t_summary = tf.summary.scalar('train_loss', loss)


        with tf.name_scope("train"):
            optimizer = self.optimizer_class(learning_rate=self.learning_rate)
            training_op = optimizer.minimize(loss)

        with tf.name_scope("eval"):    
            if self.is_sofmax_loss == True:
                y_pred = tf.argmax(logits, axis=1, name='class_predictions')
                y_proba = tf.nn.softmax(logits, name='probability_predictions')    
                y_pred_correct = tf.equal(y_pred, tf.argmax(y, axis=1))
            else:
                y_proba = tf.nn.sigmoid(logits)
                y_pred = tf.cast(tf.greater_equal(logits, 0), tf.int32)
                #y_pred = tf.cast(tf.greater_equal(y_proba, 0.5), tf.int32)
                y_pred_correct = tf.equal(y_pred, y)
            
            _,roc_score = tf.metrics.auc(y, y_proba)    
            accuracy = tf.reduce_mean(tf.cast(y_pred_correct, tf.float32), name='accuracy')
 
        init_global = tf.global_variables_initializer()
        init_local = tf.local_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        
        self._X_i, self._y = X_i, y
        self._y_proba, self._loss = y_proba, loss
        self._training_op = training_op
        self._accuracy, self._roc_score =  accuracy, roc_score
        self._init_global, self._init_local = init_global, init_local
        self._saver = saver
        self._loss_v_summary = loss_v_summary
        self._loss_t_summary = loss_t_summary

    def close_session(self):
        if self._session:
            self._session.close()
        pass

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        
        reset_graph()
        dnn_clf.clearModelFiles()
        
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()
        
        n_inputs = len(num_feats_idx)
        n_outputs = 1
        
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)        
            
            
        best_loss = np.infty
        epochs_without_progress = 0
        max_epochs_without_progress = 10
        eval_step = 5
        batches = (len(X) // self.batch_size)
        
        # Now train the model!
        
        self._session = tf.Session(graph=self._graph, config=tf.ConfigProto(log_device_placement=True))
        
        with self._session.as_default() as sess:
        
            if os.path.isfile(self._checkpoint_epoch_path):
                # if the checkpoint file exists, restore the model and load the epoch number
                with open(self._checkpoint_epoch_path, "rb") as f:
                    start_epoch = int(f.read())
                print("Training was interrupted. Continuing at epoch", start_epoch)
                self._saver.restore(sess, self._checkpoint_path)
            else:
                print("Training start")
                start_epoch = 0
                init = [self._init_global, self._init_local]
                sess.run(init)      
                
                
            for epoch in range(start_epoch, n_epochs):
                for iteration in range(batches):

                    X_batch, y_batch = random_batch(X, y, self.batch_size)

                    
                    if self._training is not None:
                        _d = self.get_feed_dict(True, cat_feats_idx, self.cat_placeholders, num_feats_idx, X_batch, y_batch)            
                    else:
                        _d = self.get_feed_dict(None, cat_feats_idx, self.cat_placeholders, num_feats_idx, X_batch, y_batch)            
                        
                    sess.run(self._training_op, feed_dict=_d)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=_d)                    
                    

                loss_train_summary_str = sess.run([self._loss_t_summary], feed_dict=_d)

                if X_valid is not None and y_valid is not None:
                    if self._training is not None:
                        _d = self.get_feed_dict(False, cat_feats_idx, self.cat_placeholders, num_feats_idx, X_valid, y_valid) 
                    else:
                        _d = self.get_feed_dict(None, cat_feats_idx, self.cat_placeholders, num_feats_idx, X_valid, y_valid) 

                    loss_val, y_proba_val, loss_summary_str = sess.run([self._loss, 
                                                                                   self._y_proba, 
                                                                                   self._loss_v_summary], 
                                                                              feed_dict=_d)
                    roc_val = self._roc_score.eval(feed_dict=_d)
                    v_conf_val = y_proba_val * 2 - 1
                    error_lb = getValidationError(v_conf_val,v_r,v_u,v_d)

                if epoch % eval_step == 0:
                    print("Epoch:", epoch,
                          "\tLoss: {:.5f}".format(loss_val),
                          "\tError:{:.5f}".format(error_lb),
                          "\tROC: {:.5f}".format(roc_val))            

                    self._saver.save(sess, self._checkpoint_path)

                    with open(self._checkpoint_epoch_path, "wb") as f:
                        f.write(b"%d" % (epoch + 1))

                    if loss_val < best_loss:
                        self._saver.save(sess, self._final_model_path)
                        best_loss = loss_val
                        epochs_without_progress = 0;
                    else:
                        epochs_without_progress += eval_step
                        if epochs_without_progress > max_epochs_without_progress:
                            print("Early stopping")
                            break

                    print('epochs_without_progress', epochs_without_progress, 'best_loss', best_loss)  
                
            #self._file_writer.flush()
            #self._file_writer.close()  

            self._saver.restore(sess, self._final_model_path)
        
        return self
                
    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
        
            if self._training is not None:
                _d = self.get_feed_dict(False, cat_feats_idx, self.cat_placeholders, num_feats_idx, X)        
            else:
                _d = self.get_feed_dict(None, cat_feats_idx, self.cat_placeholders, num_feats_idx, X)        
        
            return self._y_proba.eval(feed_dict=_d)
        pass
        
    def predict(self, X):        
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
        
            if self._training is not None:
                _d = self.get_feed_dict(False, cat_feats_idx, self.cat_placeholders, num_feats_idx, X)        
            else:
                _d = self.get_feed_dict(None, cat_feats_idx, self.cat_placeholders, num_feats_idx, X)        
        
            return self._y_pred.eval(feed_dict=_d) 
        pass
    
    def clearModelFiles(self):
        if os.path.exists('./tmp'):
            shutil.rmtree('./tmp')
        if os.path.exists('./final'):
            shutil.rmtree('./final') 
        pass
    
    def get_feed_dict(self, _training, cat_feats_idx, cat_placeholders, cont_feats_idx, batch_X, batch_y=None ):
        """ Return a feed dict for the graph including all the categorical features
        to embed """

        feed_dict = { }
        # Continuous X features and the labels if training run
        if _training is not None:
            feed_dict[self._training] = _training

        feed_dict[self._X_i] =  batch_X[:, cont_feats_idx]

        # Loop through the categorical features to provide values for the placeholders
        for idx, tensor in zip(cat_feats_idx, cat_placeholders):
            feed_dict[tensor] = batch_X[:, idx]        
        
        if batch_y is not None:
           # print('batch_y', batch_y.shape)
            feed_dict[self._y] = batch_y

        return feed_dict   
    
    def embed_and_attach(self,X, X_cat, cardinality, embed_size, g): 
        with self._graph.as_default():
            embedding = tf.Variable(tf.random_uniform([cardinality, embed_size], -1.0, 1.0))
            embedded_x = tf.nn.embedding_lookup(embedding, X_cat) 
            return tf.concat([embedded_x, X], axis=1)
        
    def score(self, X, y=None):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
        
            if self._training is not None:
                _d = self.get_feed_dict(False, cat_feats_idx, self.cat_placeholders, num_feats_idx, X,y)        
            else:
                _d = self.get_feed_dict(None, cat_feats_idx, self.cat_placeholders, num_feats_idx, X,y)        
        
            #return self._roc_score.eval(feed_dict=_d)       
            return (1 - self._loss.eval(feed_dict=_d))
        
        

In [ ]:
if val_cnt != 0:
    (tr_x, tr_y, v_x, v_y, v_u, v_r, v_d, te_x, te_y, te_u, te_d, te_r) =  trainTestSplit(all_x, all_y, market_train[:val_cnt], rs = 42)
else:
    (tr_x, tr_y, v_x, v_y, v_u, v_r, v_d, te_x, te_y, te_u, te_d, te_r) =  trainTestSplit(all_x, all_y, market_train, rs = 42)

print('tr_x.shape', tr_x.shape)
print('tr_y.shape', tr_y.shape)
print('te_x.shape', te_x.shape)
print('te_y.shape', te_y.shape)
print('v_x.shape', v_x.shape)
print('v_y.shape', v_y.shape)

In [ ]:
reset_graph()
dnn_clf = DNNClassifier(random_state=42, dropout_rate=0.5, batch_norm_momentum=0.95, n_hidden_layers = 3, 
                        n_neurons=48, batch_size = 2000)
dnn_clf.clearModelFiles()
dnn_clf.fit(tr_x, tr_y, n_epochs=1000, X_valid=v_x, y_valid=v_y)

In [ ]:
# calculation of actual metric that is used to calculate final score
pred_te = dnn_clf.predict_proba(te_x)
te_conf_val =  pred_te * 2 - 1
print('error:', getValidationError(te_conf_val,te_r,te_u,te_d))

In [ ]:
display(te_r[:2])
display(te_conf_val[:2])
display(pred_te[:2])
display(te_u[:2])

In [ ]:
def predictValues(Xp):
   # market_obs_df['assetCode'] = market_obs_df['assetCode'].apply(lambda x: x if x in unic_assetsCode else unic_assetsCode[0])
    Xp['assetCode'] = Xp['assetCode'].apply(lambda x: x if x in unic_assetsCode else unic_assetsCode[0])
    _Xp = full_pipeline.transform(Xp)
    pred_v = dnn_clf.predict_proba(_Xp)
    pred_inv = pred_v * 2 - 1
    return pred_inv

def make_random_predictions(predictions_df, Xp):
    predictions_df.confidenceValue = predictValues(Xp)

In [ ]:
#days = env.get_prediction_days()
#(market_obs_df, news_obs_df, predictions_template_df) = next(days)

In [ ]:
#make_random_predictions(predictions_template_df, market_obs_df)


In [ ]:
#predictions_template_df.head(10)

In [ ]:
#scaler.inverse_transform(predictions_template_df.confidenceValue.values.reshape(-1,1))[:5]

In [ ]:
#scaler.inverse_transform(v_y)[:5]

In [ ]:
#v_y[:5]

In [ ]:
# You can only iterate through a result from `get_prediction_days()` once
# so be careful not to lose it once you start iterating.
days = env.get_prediction_days()
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    market_train = join_market_news(market_obs_df, news_obs_df)
    make_random_predictions(predictions_template_df, market_train)
    env.predict(predictions_template_df)
print('Done!')

In [ ]:
env.write_submission_file()